In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer, util
import torch, json
import re
import time
from tqdm import tqdm

In [ ]:
!huggingface-cli login --token ""


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Project2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Project2`


In [43]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_auth_token=True,
    force_download=True,
    resume_download=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="bfloat16",
    use_auth_token=True,
    force_download=True,
    resume_download=True
)


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [44]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="bfloat16"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [45]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,)

Device set to use cuda:0


In [46]:
with open("clean_text", "r", encoding="utf-8") as f:
    text = f.read()

text = re.sub(r"\s+", " ", text).strip()

In [47]:
meaningful_verbs = {
    "fund", "finance", "grant", "invest", "support", "sponsor", "award",
    "partner", "collaborate", "work", "team", "ally", "join",
    "create", "launch", "develop", "establish", "build", "initiate",
    "set up", "start", "found", "open", "introduce",
    "provide", "offer", "deliver", "train", "educate", "enable",
    "accelerate", "incubate", "scale", "facilitate", "connect",
    "select", "mentor", "coach", "coordinate", "manage",
    "operate", "organize", "implement", "run", "mobilize",
    "campaign", "advocate", "host", "innovate", "research", "pilot", "test"
}
verbs_list = ", ".join(sorted(meaningful_verbs))

In [48]:
model_name = "sentence-transformers/all-mpnet-base-v2"
embedder = SentenceTransformer(model_name)

In [49]:
canonical_verbs = sorted(list(meaningful_verbs))
canonical_embeddings = embedder.encode(
    canonical_verbs,
    convert_to_tensor=True,
    normalize_embeddings=True
)

In [50]:
def normalize_verb(phrase, threshold = 0.3):

    new_embedding = embedder.encode(
        phrase,
        convert_to_tensor=True,
        normalize_embeddings=True
    )

    scores = util.cos_sim(new_embedding, canonical_embeddings)

    best_idx = torch.argmax(scores).item()
    best_verb = canonical_verbs[best_idx]
    best_score = scores[0][best_idx].item()


    if best_score < threshold:
      return None, best_score

    return best_verb, best_score

In [51]:
# phrase = "NetApp guided the young enterprenuers to start a tech start-up"
# best_verb, best_score = normalize_verb(phrase)

# print("Original:", phrase)
# print("Normalized:", best_verb)
# print("Similarity score:", best_score * 100)


In [52]:
def chunk_text(text, max_words=900):
    chunks = []
    words = text.split()
    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i + max_words])
        chunks.append(chunk)
    return chunks

In [53]:
with open("clean_text", "r", encoding="utf-8") as f:
    clean_text = f.read()

chunks = chunk_text(clean_text, max_words=900)

In [54]:
def make_prompt(text_chunk):
    return f"""
You are an expert in extracting organizational relationship triples.

Focus ONLY on verbs related to institutional actions like:
{', '.join(canonical_verbs)}

Each triple must follow this format:
Role: [organization taking action]
Practice: [main institutional action verb — must come from the above list or its semantic equivalent]
Counterrole: [partner or recipient organization]
Context: [short quote from the text supporting the relation]

Ignore vague or non-relational verbs (e.g. "discusses", "mentions", "focuses on").

TEXT:
{text_chunk}
"""

In [55]:
output_path = "eit_triples_local_gemma.json"
all_triples = []

In [56]:
for i, chunk in enumerate(tqdm(chunks, desc="Generating triples")):
    prompt = make_prompt(chunk)

    try:
        result = pipe(
            prompt,
            max_new_tokens=400,
            temperature=0.3,
            top_p=0.9,
            return_full_text=False
        )

        generated = result[0]["generated_text"].strip()

        normalized_output = []
        for line in generated.splitlines():
            if line.lower().startswith("practice:"):
                verb_phrase = line.split(":", 1)[1].strip()
                normalized = normalize_verb(verb_phrase)
                if normalized:
                    line = f"Practice: {normalized}"
            normalized_output.append(line)

        normalized_text = "\n".join(normalized_output)

        all_triples.append({"chunk_id": i + 1, "text": normalized_text})

        if i % 3 == 0:
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(all_triples, f, ensure_ascii=False, indent=2)

        time.sleep(0.5)

    except Exception as e:
        print(f"Error in chunk {i + 1}: {e}")
        continue

Generating triples: 100%|██████████| 82/82 [12:06<00:00,  8.86s/it]


In [57]:
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_triples, f, ensure_ascii=False, indent=2)

In [58]:
import spacy

try:
    nlp = spacy.load("en_core_web_trf")
    print("Loaded spaCy transformer model (en_core_web_trf).")
except OSError:
    nlp = spacy.load("en_core_web_sm")
    print("Transformer model not found so using en_core_web_sm instead")

Transformer model not found so using en_core_web_sm instead


In [60]:
with open("eit_triples_local_gemma.json", "r", encoding="utf-8") as f:
    raw_triples = json.load(f)

In [61]:
triple_pattern = re.compile(
    r"Role:\s*(.+?)\s*"
    r"Practice:\s*(.+?)\s*"
    r"Counterrole:\s*(.+?)\s*"
    r"Context:\s*(.+?)(?=Role:|$)",
    re.DOTALL | re.IGNORECASE
)

In [62]:
rich_triples = []

for entry in tqdm(raw_triples, desc="Enriching triples"):
    text = entry.get("text", "")
    matches = triple_pattern.findall(text)

    for role, practice, counterrole, context in matches:
        context = context.strip()

        # spacy NER on the context
        doc = nlp(context)
        ner_data = [
            {"text": ent.text, "label": ent.label_}
            for ent in doc.ents
            if ent.label_ in {"ORG", "INSTITUTION", "GPE"}
        ]

        # Clean up whitespace
        rich_triples.append({
            "role": role.strip(),
            "practice": practice.strip(),
            "counterrole": counterrole.strip(),
            "context": context,
            "ner": ner_data
        })


Enriching triples: 100%|██████████| 82/82 [00:03<00:00, 27.27it/s]


In [63]:
output_path = "eit_rich_triples_gemma.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(rich_triples, f, ensure_ascii=False, indent=2)

print(f"Created {len(rich_triples)} rich triples with NER.")
print(f"Saved to {output_path}")

Created 295 rich triples with NER.
Saved to eit_rich_triples_gemma.json


In [64]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("CEO of EIT Digital A central element of this partnership is the launch of the National Dual-Use Technology Accelerator")
spacy.displacy.serve(doc, style="dep")

/usr/local/lib/python3.12/dist-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
